In [1]:
keyword = 'chris evans'
date = ('2011-06-22', '2011-08-22')
page_scroll = 5
username = 'qitianma'
password = 'qitian1129'

In [5]:
def twitter_scrape(keyword, date, page_scroll, username, password):
    """
    Please CHANGE your ChromeDriver executable_path!!!
    
    Parameters
    ----------
    (keyword = int, date = tuple, pageroll = int, username = str, password = str) -> data = pd.DataFrame
    
    keyword: the word you want to search
    date: date[0] is the starting date, date[1] is the end date
    page_scroll: the number of pages you want to scroll (the function must see the whole content before scraping)
    username: your twitter username
    password: your twitter password
    
    data: fetched data
    """

    SCROLL_PAUSE_TIME = 2

    from selenium import webdriver
    from bs4 import BeautifulSoup
    import time
    import pandas as pd

    # Open Browser (Please CHANGE your ChromeDriver executable_path!!!)
    driver = webdriver.Chrome(executable_path = '/usr/bin/chromedriver')
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"}

    # Log in Twitter Account
    url = 'https://twitter.com/login'
    driver.get(url)
    username_box = driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > fieldset > div:nth-child(2) > input')
    password_box = driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > fieldset > div:nth-child(3) > input')
    username_box.send_keys(username)
    password_box.send_keys(password)
    driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > div.clearfix > button').click()

    # Search Keyword
    search_entry = keyword + ' since:' + date[0] + ' until:' + date[1]
    search_box = driver.find_element_by_css_selector('#search-query')
    search_box.send_keys(search_entry)
    driver.find_element_by_css_selector('#global-nav-search > span > button').click()

    # Scroll Pages
    for page in range(page_scroll):
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Fetch Webpage & Parse Content
    data = pd.DataFrame({'content': [], 
                         'usr': [], 
                         'account_id': [], 
                         'timestamp': [], 
                         'comment': [], 
                         'retweet': [], 
                         'favorite': [], 
                         'link': []})
    souped_content = BeautifulSoup(driver.page_source)
    driver.quit()
    
    # Extract Data
    streams = souped_content.find_all('div', {'class': 'content'})
    for stream in streams:
        try:
            header = stream.find('div', {'class': 'stream-item-header'})
            if header is None:
                continue
            usr = header.find('span', {'class': "FullNameGroup"}).text.strip()
            account_id = header.find('a', {'class': "account-group js-account-group js-action-profile js-user-profile-link js-nav"}).attrs['data-user-id']
            timestamp = pd.to_datetime(header.find('span', {'class': "_timestamp js-short-timestamp "}).attrs['data-time-ms'], unit='ms')
            link = header.find('a', {'class': "tweet-timestamp js-permalink js-nav js-tooltip"}).attrs['href']

            text_container = stream.find('div', {'class': 'js-tweet-text-container'})
            if text_container.find('p', {'class', 'TweetTextSize js-tweet-text tweet-text'}).attrs['lang'] != 'en':
                continue
            content = text_container.text.strip()

            footer = stream.find('div', {'class': 'stream-item-footer'})
            statistics = footer.find_all('span', {'class': 'ProfileTweet-actionCount'})
            try: 
                comment =  statistics[0].attrs['data-tweet-stat-count']
            except:
                comment = 0
            try:
                retweet =  statistics[1].attrs['data-tweet-stat-count']
            except:
                retweet = 0
            try:
                favorite =  statistics[2].attrs['data-tweet-stat-count']
            except:
                favorite = 0
            data = data.append({'content': content, 
                                'usr': usr, 
                                'account_id': account_id, 
                                'timestamp': timestamp, 
                                'comment': comment, 
                                'retweet': retweet, 
                                'favorite': favorite, 
                                'link': link},
                                ignore_index=True)
        except:
            continue
    return data

In [6]:
twitter_scrape?

Signature: twitter_scrape(keyword, date, page_scroll, username, password)
Docstring:
Please CHANGE your ChromeDriver executable_path!!!

Parameters
----------
(keyword = int, date = tuple, pageroll = int, username = str, password = str) -> data = pd.DataFrame

keyword: the word you want to search
date: date[0] is the starting date, date[1] is the end date
page_scroll: the number of pages you want to scroll (the function must see the whole content before scraping)
username: your twitter username
password: your twitter password

data: fetched data
File:      ~/Documents/NLP/Code/<ipython-input-5-0f4c05726a6a>
Type:      function


In [3]:
def twitter_scrape_byday(keyword, date, page_scroll, username, password):
    from selenium import webdriver
    from bs4 import BeautifulSoup
    import time
    import pandas as pd
    
    date_range = pd.date_range(date[0], date[1], freq = 'D')
    date_range = [str(date).split()[0] for date in date_range]
    data = pd.DataFrame({'content': [], 
                         'usr': [], 
                         'account_id': [], 
                         'timestamp': [], 
                         'comment': [], 
                         'retweet': [], 
                         'favorite': [], 
                         'link': []})
    
    SCROLL_PAUSE_TIME = 2

    # Open Browser (Please CHANGE your ChromeDriver executable_path!!!)
    driver = webdriver.Chrome(executable_path = '/usr/bin/chromedriver')
    headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36"}

    # Log in Twitter Account
    url = 'https://twitter.com/login'
    driver.get(url)
    username_box = driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > fieldset > div:nth-child(2) > input')
    password_box = driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > fieldset > div:nth-child(3) > input')
    username_box.send_keys(username)
    password_box.send_keys(password)
    driver.find_element_by_css_selector('#page-container > div > div.signin-wrapper > form > div.clearfix > button').click()
    
    for date_idx in range(len(date_range) - 1):
        date_tup = (date_range[date_idx], date_range[date_idx+1]) 
        search_entry = keyword + ' since:' + date_tup[0] + ' until:' + date_tup[1]
        search_box = driver.find_element_by_css_selector('#search-query')
        search_box.clear()
        search_box.send_keys(search_entry)
        driver.find_element_by_css_selector('#global-nav-search > span > button').click()

        # Scroll Pages
        for page in range(page_scroll):
            time.sleep(SCROLL_PAUSE_TIME)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Fetch Webpage & Parse Content
        souped_content = BeautifulSoup(driver.page_source)

        # Extract Data
        streams = souped_content.find_all('div', {'class': 'content'})
        for stream in streams:
            try:
                header = stream.find('div', {'class': 'stream-item-header'})
                if header is None:
                    continue
                usr = header.find('span', {'class': "FullNameGroup"}).text.strip()
                account_id = header.find('a', {'class': "account-group js-account-group js-action-profile js-user-profile-link js-nav"}).attrs['data-user-id']
                timestamp = pd.to_datetime(header.find('span', {'class': "_timestamp js-short-timestamp "}).attrs['data-time-ms'], unit='ms')
                link = header.find('a', {'class': "tweet-timestamp js-permalink js-nav js-tooltip"}).attrs['href']

                text_container = stream.find('div', {'class': 'js-tweet-text-container'})
                if text_container.find('p', {'class', 'TweetTextSize js-tweet-text tweet-text'}).attrs['lang'] != 'en':
                    continue
                content = text_container.text.strip()

                footer = stream.find('div', {'class': 'stream-item-footer'})
                statistics = footer.find_all('span', {'class': 'ProfileTweet-actionCount'})
                try: 
                    comment =  statistics[0].attrs['data-tweet-stat-count']
                except:
                    comment = 0
                try:
                    retweet =  statistics[1].attrs['data-tweet-stat-count']
                except:
                    retweet = 0
                try:
                    favorite =  statistics[2].attrs['data-tweet-stat-count']
                except:
                    favorite = 0
                data = data.append({'content': content, 
                                    'usr': usr, 
                                    'account_id': account_id, 
                                    'timestamp': timestamp, 
                                    'comment': comment, 
                                    'retweet': retweet, 
                                    'favorite': favorite, 
                                    'link': link},
                                    ignore_index=True)
            except:
                continue
    driver.quit()
    return data

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
sid = SentimentIntensityAnalyzer()

In [4]:
data = twitter_scrape_byday(keyword, date, page_scroll, username, password)

In [9]:
data2 = data.copy()

In [15]:
data2['polarity'] = data2['content'].apply(lambda x: sid.polarity_scores(x)['compound'])

In [20]:
data2['date'] = data2['timestamp'].apply(lambda x: str(x).split()[0])

In [28]:
import numpy as np

In [31]:
po = data2.groupby('date').apply(np.mean)

In [39]:
po

,polarity
date,
2011-06-22,0.004979
2011-06-23,0.031122
2011-06-24,0.187331
2011-06-25,0.162548
2011-06-26,0.136949
2011-06-27,0.176067
2011-06-28,0.110033
2011-06-29,0.141255
2011-06-30,0.143219


In [69]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: TkAgg


In [72]:
plt.plot(pd.to_datetime(po['polarity'].index[:-1]), po['polarity'].values[:-1])

In [71]:
pd.to_datetime(po['polarity'].index)[:-1]

DatetimeIndex(['2011-06-22', '2011-06-23', '2011-06-24', '2011-06-25',
               '2011-06-26', '2011-06-27', '2011-06-28', '2011-06-29',
               '2011-06-30', '2011-07-01', '2011-07-02', '2011-07-03',
               '2011-07-04', '2011-07-05', '2011-07-06', '2011-07-07',
               '2011-07-08', '2011-07-09', '2011-07-10', '2011-07-11',
               '2011-07-12', '2011-07-13', '2011-07-14', '2011-07-15',
               '2011-07-16', '2011-07-17', '2011-07-18', '2011-07-19',
               '2011-07-20', '2011-07-21', '2011-07-22', '2011-07-23',
               '2011-07-24', '2011-07-25', '2011-07-26', '2011-07-27',
               '2011-07-28', '2011-07-29', '2011-07-30', '2011-07-31',
               '2011-08-01', '2011-08-02', '2011-08-03', '2011-08-04',
               '2011-08-05', '2011-08-06', '2011-08-07', '2011-08-08',
               '2011-08-09', '2011-08-10', '2011-08-11', '2011-08-12',
               '2011-08-13', '2011-08-14', '2011-08-15', '2011-08-16',
      

In [66]:
pd.to_datetime?

Signature: pd.to_datetime(arg, errors='raise', dayfirst=False, yearfirst=False, utc=None, box=True, format=None, exact=True, unit=None, infer_datetime_format=False, origin='unix', cache=False)
Docstring:
Convert argument to datetime.

Parameters
----------
arg : integer, float, string, datetime, list, tuple, 1-d array, Series

    .. versionadded:: 0.18.1

       or DataFrame/dict-like

errors : {'ignore', 'raise', 'coerce'}, default 'raise'

    - If 'raise', then invalid parsing will raise an exception
    - If 'coerce', then invalid parsing will be set as NaT
    - If 'ignore', then invalid parsing will return the input
dayfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.
    If True, parses dates with the day first, eg 10/11/12 is parsed as
    2012-11-10.
    with day first (this is a known bug, based on dateutil behavior).
yearfirst : boolean, default False
    Specify a date parse order if `arg` is str or its list-likes.

    - If T

In [13]:
sid.polarity_scores(data.loc[1, 'content'])

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}